# Sisendtekstide valideerimine ja puhastamine


[![License: GPLv2](https://img.shields.io/badge/License-GPL%20v2-lightgrey.svg)](https://www.gnu.org/licenses/old-licenses/gpl-2.0.en.html)

Tekstide puhastamiseks ja valideerimiseks on mitmeid viise. Kõige parema tulemuse annab EstNLTK teegi teenuste kasutamine tekstide jagamisel sõnadeks. Selle teegi kasutamisel peavad skriptid ise olema litsenseeritud GPL v2 litsentsi alusel. See paneb piirangud vaid analüüsikoodi edasisele integreerimisele mitte analüüsitulemustele. Kui kood on pakendatud veebiteenuseks või kutsutakse välja käsurealt, siis seda väljakutsuv programm võib olla ükskõik millise litsentsiga.  


In [1]:
import re
import math
import time
import requests
import datetime
import urllib.parse

from datetime import date
from pandas import DataFrame 
from pandas import read_csv
from pandas import concat
from tqdm.auto import tqdm

from estnltk import Text
from estnltk.taggers import VabamorfAnalyzer
from estnltk.taggers import PostMorphAnalysisTagger
from estnltk.taggers.standard.morph_analysis.proxy import MorphAnalyzedToken

from typing import List

## I. EstNLTK kasutamine sõnade algvormi leidmiseks

EstNLTK võimaldab automaatselt teksti tükeldada sõnadeks ning seejärel leida sellele vastava algvormi. 
Kuigi EstNLTK võimaldab välistada sõna algvormide hulgast konteksti mitte sobituvad (mürgise mao → madu mitte magu), siis teeb selline ühestamine vahel vigu ja otsingu kontekstis on kasulik säilitada kõikvõimalikud vormid. 

Skriptis [`02A_validate_and_clean_source_texts_naively.ipynb`](./02A_validate_and_clean_source_texts_naively.ipynb) saadud analüüsi kordamiseks on meil vaja funktsiooni tekstide puhastamiseks ja normaliseerimiseks ning sellega kooskõlalist funktsiooni algvormide leidmiseks, mille väljund oleks analoogne originaalskriptiga.


In [2]:
# Single morph analyser with post-processing adjustments for the entire code
# These together do not perform disambiguation.
MORPH_ANALYZER = VabamorfAnalyzer()
POST_MORPH_ANALYZER = PostMorphAnalysisTagger()

def normalise_text(text: str) -> List[str]:  
    """
    Returns a list of tokens each corresponding to a word.

    Direct call to EstNLTK tokeniser that uses the best common sense tricks for tokenization
    """
    return ' '.join(word.text for word in Text(text).tag_layer('words')['words'])

def analyze_text(text: str):
    """
    Returns a four column table with columns index, wordform, lemma, sublemmas.
    There can be several rows for each word as each word is analysed separately.
    All rows with the same index correspond to the same word.
    Wordform columns is added to facilitate tokenisation debugging
    """
    morph_layer = POST_MORPH_ANALYZER.retag(MORPH_ANALYZER.tag(Text(text).tag_layer(['words','sentences'])))['morph_analysis']
    return (
        DataFrame(
            [[i, span.text, lemma, tuple(subwords)] 
             for i, span in enumerate(morph_layer) 
             for  lemma, subwords in zip(span['lemma'], span['root_tokens'])],
            columns = ['index', 'wordform','lemma','sublemmas'])
        .drop_duplicates()
        .assign(sublemmas=lambda df: df['sublemmas'].map(list)))

In [3]:
# Example outputs
print(normalise_text('Presidendi ametiraha seadus'))
print(normalise_text('Rahvusvahelise tööorganisatsiooni (ILO) konventsiooni "Mees- ja naistöötajate ... töö eest" ratifitseerimise seadus'))
display(analyze_text('Presidendi ametiraha seadus'))

Presidendi ametiraha seadus
Rahvusvahelise tööorganisatsiooni ( ILO ) konventsiooni " Mees- ja naistöötajate ... töö eest " ratifitseerimise seadus


,index,wordform,lemma,sublemmas
0,0,Presidendi,President,[President]
1,0,Presidendi,Presidend,"[Pre, sidend]"
2,0,Presidendi,president,[president]
3,1,ametiraha,ametiraha,"[ameti, raha]"
6,2,seadus,seaduma,[seadu]
7,2,seadus,seadus,[seadus]


### II. Sisendtekstide analüüs

In [4]:
sources = {}
sources['state_laws'] = read_csv('../results/source_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/source_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/source_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/source_texts/government_orders.csv', header=0)

all_documents = concat([source for _,source in sources.items()], axis=0)

In [5]:
result = [None] *  len(all_documents)
for i, caption in tqdm(enumerate(all_documents['document_title']), total=len(all_documents)):
    result[i] = analyze_text(caption).assign(doc_id = i)

result = concat(result, axis=0)
display(result)

  0%|          | 0/58837 [00:00<?, ?it/s]

,index,wordform,lemma,sublemmas,doc_id
0,0,Ajutise,Ajutine,[Ajutine],0
1,0,Ajutise,Ajutis,[Ajutis],0
2,0,Ajutise,ajutine,[ajutine],0
3,1,sisseveo,sissevedu,"[sisse, vedu]",0
4,2,konventsiooniga,konventsioon,[konventsioon],0
...,...,...,...,...,...
3,0,Jõe,jõgi,[jõgi],58836
4,1,paadisadama,paadisadam,"[paadi, sadam]",58836
5,2,akvatooriumi,akvatoorium,[akvatoorium],58836
8,3,piiride,piir,[piir],58836


Otsime anomaaliad analüüsides esmalt sõna algvorme (lemmasid). Kõigepealt vaatleme harva esinevaid algvorme.

In [6]:
lemma_counts = (result
                .groupby('lemma').agg(occurence_count=('lemma', len), document_count = ('doc_id', lambda x: len(set(x))))
                .sort_values(['occurence_count', 'document_count'], ascending=False).reset_index()
                .pipe(lambda df: df[df['lemma'].str.len() > 0])
               )
display(lemma_counts.tail(15))

,lemma,occurence_count,document_count
23256,ümberregistreerimata,1,1
23257,ümberregistreerimine,1,1
23258,ümbertegemine,1,1
23259,üürikorter,1,1
23260,üüritariif,1,1
23261,üüritoetus,1,1
23262,üüriturg,1,1
23263,üürivaidlus,1,1
23264,ˮEuroopa,1,1
23265,ˮinimõigus,1,1


**Tulemus:** EstNLTK sõnestaja ei suuda eraldada kõiki UTF8 kirjavahemärke tekstist (erikujulised jutumärgid).

Vaatame erandlikke sõnu lähemalt jättes alles kõik need sõnad, mis sisaldavad eesti tähestikku mitte kuuluvaid sümboleid.

In [7]:
ESTONIAN_LETTER = '[a-z|öäõü|\-|žš]'
ESTONIAN_LETTERS_ONLY = f'^(?:{ESTONIAN_LETTER})+$' 

In [8]:
incorrect = ~lemma_counts['lemma'].str.contains(ESTONIAN_LETTERS_ONLY, case=False, regex=True)
display(concat([
    lemma_counts[incorrect]
    .head(15)
    .reset_index(drop=True),
    lemma_counts[incorrect]
    .tail(15)
    .reset_index(drop=True)], axis=1))

,lemma,occurence_count,document_count,lemma,occurence_count,document_count
0,",",6491,4080,m²,1,1
1,„,957,826,nimekiri1,1,1
2,"""",886,365,osaˮ,1,1
3,2020,842,804,rakendamine1,1,1
4,“,828,733,tingimused1,1,1
5,2017.,655,652,vormid1,1,1
6,2017,613,605,¿,1,1
7,2021,608,603,Äriseadustik1,1,1
8,”,596,482,ˮEuroopa,1,1
9,2019,495,486,ˮinimõigus,1,1


**Tulemus:** Lemmade hulgas on palju aastanumbreid nagu ka originaalanalüüsis.

In [9]:
NUMBER_SYMBOLS = '[0-9]'
NUMBERS_ONLY = f'^{NUMBER_SYMBOLS}+$'

numbers = lemma_counts['lemma'].str.contains(NUMBERS_ONLY, case=False, regex=True)
unanalysed = incorrect & ~ numbers
display(lemma_counts[unanalysed & lemma_counts['lemma'].str.contains(f'{NUMBER_SYMBOLS}+')])

,lemma,occurence_count,document_count
117,2017.,655,652
211,2018.,381,381
218,2019.,376,374
223,2020.,369,366
228,2021.,363,361
...,...,...,...
20989,nimekiri1,1,1
21522,rakendamine1,1,1
22367,tingimused1,1,1
22888,vormid1,1,1


**Tulemus:** Samas on EstNLTK lahutanud numbrivahemikud väiksemateks sõnedeks, kuid järgarvud on ikka ühe sõnena sees. See raskendab otsingut.

In [10]:
DASH_SYMBOLS = '[-−‒]'
SLASH_SYMBOLS = '[/]'
NUMBER = f'{NUMBER_SYMBOLS}+'
NUMBER_RANGE = f'(?:(?:{NUMBER})?{DASH_SYMBOLS}{NUMBER})|(?:{NUMBER}{DASH_SYMBOLS})|(?:{NUMBER}{SLASH_SYMBOLS}{NUMBER})'
NUMBER_RANGE_ONLY = f'^(?:{NUMBER_RANGE})$'

number_range = lemma_counts['lemma'].str.contains(NUMBER_RANGE_ONLY)
assert re.match(NUMBER_RANGE_ONLY, '-2012') is not None
assert re.match(NUMBER_RANGE_ONLY, '1930-2023') is not None
assert re.match(NUMBER_RANGE_ONLY, '2017/2018') is not None
assert re.match(NUMBER_RANGE_ONLY, '−2025') is not None
assert re.match(NUMBER_RANGE_ONLY, '2017-') is not None
assert re.match(NUMBER_RANGE_ONLY, '2014‒2020') is not None
assert not any(number_range), 'Ootamatu numbrivahemik' 

ordinals = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER})+\.$')
unanalysed = incorrect & ~ numbers & ~number_range & ~ordinals


**Tulemus:** Erinevalt naiivsest tokenisatsioonist puuduvad meil numbrivahemikud, kuid järgarvud on ikka ühe sõnana sees.

In [11]:
lemma_counts[unanalysed & (lemma_counts['lemma'].str.len() == 1)]

,lemma,occurence_count,document_count
6,",",6491,4080
59,„,957,826
66,"""",886,365
74,“,828,733
137,”,596,482
237,»,343,275
238,«,340,274
299,/,280,217
320,(,262,241
323,),261,240


**Tulemus:** EstNLTK sõnestus sisaldab oluliselt rohkem kirjavahemärke ja kahte erikujulist tühikusümbolit (*whitespace*). 

In [12]:
DASH_SYMBOLS = '[‑-−‒-]'
PUNCTUATION_MARKS = '[\\.,:;!?¿\\(\\)«»„““ˮ"‟”\[\]]'
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'
OTHER_SYMBOLS = '[§/−a\^=+\*]'

In [13]:
idx = lemma_counts['lemma'].str.contains(f'^(?:{DASH_SYMBOLS}|{PUNCTUATION_MARKS}|{OTHER_SYMBOLS}|{WHITESPACE_SYMBOLS})')
assert not any(unanalysed & (lemma_counts['lemma'].str.len() == 1) & ~idx), 'Mõni üksiksümbol on katmata'
unanalysed &= ~ idx

display(lemma_counts[unanalysed & lemma_counts['lemma'].str.contains(NUMBER)])

,lemma,occurence_count,document_count
753,COVID-19,105,105
3411,2.3,14,14
4859,1.1,8,8
4860,10.00,8,8
5341,2.1,7,7
...,...,...,...
20989,nimekiri1,1,1
21522,rakendamine1,1,1
22367,tingimused1,1,1
22888,vormid1,1,1


**Tulemus:** EstNLTK leiab ülesse sektsioonide numeratsiooni, kümnendmurrud ja kuupäevad.

In [14]:
num_dot_expressions = lemma_counts['lemma'].str.contains(f'^(?:(?:{NUMBER})+\.)+(?:{NUMBER})*$')
display(lemma_counts[unanalysed & num_dot_expressions])
unanalysed &= ~ num_dot_expressions

decimals_fractions = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER})+,(?:{NUMBER})+$')
lemma_counts[unanalysed & decimals_fractions]
unanalysed &= ~ decimals_fractions

,lemma,occurence_count,document_count
3411,2.3,14,14
4859,1.1,8,8
4860,10.00,8,8
5341,2.1,7,7
5344,4.6,7,7
...,...,...,...
14947,3.9,1,1
14949,30.01.2009,1,1
14950,30.11.2016,1,1
14951,31.12.2017,1,1


In [15]:
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
idx = lemma_counts['lemma'].str.contains(f'{SUPERSCRIPT_SYMBOLS}|{SUBSCRIPT_SYMBOLS}', regex=True)
display(lemma_counts[idx])
unanalysed &= ~idx

,lemma,occurence_count,document_count
10860,23¹,2,2
10875,3¹,2,2
14859,13¹,1,1
14869,16¹,1,1
14908,22¹,1,1
14911,23²,1,1
14916,24²,1,1
14993,46⁷,1,1
14998,4⁴,1,1
15033,67³,1,1


**Tulemus:** EstNLTK ei eralda UTF8 superskripti sümboleid sõnadest.

In [16]:
PERCENT_SYMBOLS ='[%]'
percents = lemma_counts['lemma'].str.contains(f'{PERCENT_SYMBOLS}')
display(lemma_counts[unanalysed & percents])
unanalysed &= ~percents


,lemma,occurence_count,document_count
8863,5%,3,3
10836,0%,2,2
10878,40%,2,2
15057,9%,1,1


**Tulemus:** EstNLTK ei lahuta protsente numbritest.

Allesjäänud numbreid sisaldavatest sõnedest on enamik tüüpi `10kroonine` või `kord1` ja kõik ülejäänud on erinevad lühendid või sümbolikombinatsioonid.

In [17]:
idx1 = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER})+(?:{ESTONIAN_LETTER})+$')
display(lemma_counts[unanalysed & idx1])
unanalysed &= ~idx1  

idx2 = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER})+(?:{NUMBER})+$')
display(lemma_counts[unanalysed & idx2])
unanalysed &= ~idx2

,lemma,occurence_count,document_count
8867,7aastane,3,3
10841,10kroonine,2,2
10844,12meetrine,2,2
10855,19aastane,2,2
10861,24meetrine,2,2
10888,500kroonine,2,2
10891,5aastane,2,2
14834,1000ruutmeetrine,1,1
14835,100kroonine,1,1
14895,1a,1,1


,lemma,occurence_count,document_count
7021,kord1,5,5
18914,eeskiri1,1,1
19842,kinnitamine1,1,1
19911,kohta1,1,1
20177,kvaliteedinõuded1,1,1
20278,ladustamisega1,1,1
20600,lõppseadmele1,1,1
20989,nimekiri1,1,1
21522,rakendamine1,1,1
22367,tingimused1,1,1


In [18]:
idx = lemma_counts['lemma'].str.contains(f'{NUMBER}')
display(lemma_counts[unanalysed & idx])
unanalysed &= ~idx

,lemma,occurence_count,document_count
753,COVID-19,105,105
6763,SARS-CoV-2,5,5
8852,2020ˮ,3,3
9285,NO99,3,3
10842,11/12/14,2,2
10856,2013ˮ,2,2
10886,4S2,2,2
14840,10а,1,1
14978,3M,1,1
14982,43A,1,1


In [19]:
display(lemma_counts[unanalysed])

,lemma,occurence_count,document_count
2582,F. J. Wiedemann,21,21
7471,C. R. Jakobson,4,4
7528,J.V. veski,4,4
7624,M. Lüdigi,4,4
8868,A. H. Tammsaare,3,3
8931,Côte,3,3
8982,Fr. R. Kreutzwald,3,3
9026,J. V. veski,3,3
10994,Brasília,2,2
11658,O. luts,2,2


**Tulemus:** Järele jäänud sõnad on pärisnimed ja lisajutumärkidest tekkinud vead.

## III. EstNLTK tokenisaatori parandamine

Osa leitud vigadest saab parandada EstNLTK tokenisaatori kohandamisega:
* Esialgseid sõnesid lisaks tükeldada.
* Ainult tühikusümboleid sisaldavad sõned tuleb eemaldada. 
* Osa analüüsi käigus kokku tõstetavatest sõnedest (järgarvud) tuleb tagasi lahku tõsta. 

### Esialgsete sõnede parem tükeldamine

Selleks on vaja defineerida sõnedetükledajale reeglid, mis ütlevad millal ja kust kohast sõne edasi tükledada. 

In [20]:
from estnltk_patches.local_token_splitter import LocalTokenSplitter

NUMBER = '[0-9]+'
ESTONIAN_LETTER = '[a-z|öäõü|\-|žš]'
FOREIGN_LETTER = '[ôíëаa]'
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'

**Ootamatud sümbolid sõne alguses ja lõpus**

In [21]:
idx = ~lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{NUMBER})', case=False)
idx &= lemma_counts['lemma'].str.len() > 1
display(lemma_counts[idx])

idx = ~lemma_counts['lemma'].str.contains(f'(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{NUMBER}|\.|{SUPERSCRIPT_SYMBOLS})$', case=False)
idx &= lemma_counts['lemma'].str.len() > 1
display(lemma_counts[idx])

,lemma,occurence_count,document_count
2229,.-,26,26
23264,ˮEuroopa,1,1
23265,ˮinimõigus,1,1
23266,ˮsisekokkulepe,1,1
23267,ˮüks,1,1
23269,’),1,1


,lemma,occurence_count,document_count
8852,2020ˮ,3,3
8863,5%,3,3
10836,0%,2,2
10856,2013ˮ,2,2
10878,40%,2,2
15057,9%,1,1
19913,kohtaˮ,1,1
19929,kohustusperioodilˮ,1,1
20014,koondloendˮ,1,1
21093,osaˮ,1,1


In [22]:
rule_1 = (re.compile("^(\.-|ˮ|’\))"), lambda x, y: 1)
rule_2 = (re.compile("ˮ$"), lambda x, y: y.start()) 
token_splitter = LocalTokenSplitter([rule_1, rule_2])
text = Text('ˮEuroopa ˮüks ’) .- 2020ˮ 5%').tag_layer('tokens')
token_splitter.retag(text)
text['tokens'].text

['ˮ', 'Euroopa', 'ˮ', 'üks', '’', ')', '.', '-', '2020', 'ˮ', '5', '%']

**Superskripti sümbolid sõne lõpus**

In [23]:
idx = lemma_counts['lemma'].str.contains(f'(?:{SUPERSCRIPT_SYMBOLS}|{SUBSCRIPT_SYMBOLS})$', case=False)
lemma_counts[idx]

,lemma,occurence_count,document_count
10860,23¹,2,2
10875,3¹,2,2
14859,13¹,1,1
14869,16¹,1,1
14908,22¹,1,1
14911,23²,1,1
14916,24²,1,1
14993,46⁷,1,1
14998,4⁴,1,1
15033,67³,1,1


In [24]:
CRE_NUMBER = re.compile('^[0-9]+$')

def split_if_prefix_is_word_or_number(text, match):
    return match.start() if MorphAnalyzedToken(text[0:match.start()]).is_word else -1

rule_3 = (re.compile(f'({SUPERSCRIPT_SYMBOLS}|{SUBSCRIPT_SYMBOLS})+$'), split_if_prefix_is_word_or_number)
token_splitter = LocalTokenSplitter([rule_3])
text = Text('kohta¹	67³ m²').tag_layer('tokens')
token_splitter.retag(text)
text['tokens'].text


['kohta', '¹', '67', '³', 'm²']

**Number sõna lõpus**

In [25]:
idx = lemma_counts['lemma'].str.contains(f'{ESTONIAN_LETTER}{NUMBER}$', case=False)
lemma_counts[idx]

,lemma,occurence_count,document_count
753,COVID-19,105,105
6763,SARS-CoV-2,5,5
7021,kord1,5,5
9285,NO99,3,3
10886,4S2,2,2
14997,4S3,1,1
15348,CO2,1,1
15350,CR14,1,1
15351,CT1,1,1
15352,CT2,1,1


In [26]:
def split_if_prefix_is_word(text, match):
    if CRE_NUMBER.match(text[0:match.start()]) or text[0:match.start()].isupper():
        return -1
    return match.start() if MorphAnalyzedToken(text[0:match.start()]).is_word else -1 

rule_3 = (re.compile('[0-9]+$'), split_if_prefix_is_word)
token_splitter = LocalTokenSplitter([rule_3])
text = Text('Äriseadustik1	Valga-1 RAS1000 COVID-19 NO99').tag_layer('tokens')
token_splitter.retag(text)
text['tokens'].text

['Äriseadustik', '1', 'Valga', '-', '1', 'RAS1000', 'COVID', '-', '19', 'NO99']

### Vaid tühikuid sisaldavate sõnede eemaldamine

**Tühikutest koosnevad sõned**

Sümbolid `\ufeff` ja `\u200e` lähevad sõnestamisel eraldi sõnedeks, mida edaspidises vaja ei lähe. Seega peaks analüüsi käigus vastavad sõned ära kustutama.  

In [27]:
text = Text('ehitise\ufeff\ufeff kohta\u200e').tag_layer('tokens')
bad_spans = [span for span in text['tokens'] if re.match(f'^(\s|{WHITESPACE_SYMBOLS})+$', span.text)]
for span in bad_spans:
    text['tokens'].remove_span(span)
text['tokens'].text

['ehitise', 'kohta']

### Üleliigsete liitsõnede eemaldamine

Peale sõnede loomist ühendab EstNLTK mõned sõned liitsõnedeks (näiteks kuupäevad, protsendid ja järgarvud).
Antud kontekstis tuleb meil osa neist moodustatud liitsõnedest eemaldada (protsendid ja järgarvud), et otsimine oleks efektiivsem.

In [28]:
def is_ordinal(span) -> bool:
    return (len(span) == 2) and (span[1].text == '.') and re.match('^[0-9]+$', span[0].text)

text = Text('1997. aasta 15.-17. septembril kell 14.15 tehtud otsus') 
text.tag_layer('compound_tokens')
print(list(''.join(token.text) for token in text['compound_tokens']))

for span in [span for span in text['compound_tokens'] if is_ordinal(span)]:
    text['compound_tokens'].remove_span(span)
print(list(''.join(token.text) for token in text['compound_tokens']))


['1997.', '15.', '17.', '14.15']
['14.15']


In [29]:
def is_percent(span) -> bool:
    if len(span) == 2 and span[1].text == '%':
        return re.match('^[0-9]+$', span[0].text)
    if len(span) == 4 and span[3].text == '%' and re.match('\.|,', span[1].text):
        return re.match('^[0-9]+$', span[0].text) and re.match('^[0-9]+$', span[2].text)
    return False

text = Text('Töötuse määrad 15% ja 1,05 % ning 1.1 protsenti') 
text.tag_layer('compound_tokens')
print(list(''.join(token.text) for token in text['compound_tokens']))

for span in [span for span in text['compound_tokens'] if is_percent(span)]:
    text['compound_tokens'].remove_span(span)

print(list(''.join(token.text) for token in text['compound_tokens']))

['15%', '1,05%', '1.1']
['1.1']


## IV. Parandatud EstNLTK analüüsitöövoo valideerimine

EstNLTK meetod `Text.tag_layer` kasutab vajalike analüüsitulemuste saavutamiseks standardset töövoogu, mida on võimalik globaalselt ümber seadistada. Meie kontekstis on lihtsam ja selgem defineerida eraldi klass, mis eksplisiitselt realiseerib muudetud töövoo ning selle abil täpsustada funktsiooni `analyze_text`. 

In [30]:
from estnltk_patches.rt_text_analyzer import RTTextAnalyzer

RT_MORPH_ANALYZER = RTTextAnalyzer()

def rt_normalise_text(text: str) -> List[str]:  
    """
    Returns a list of tokens each corresponding to a word.

    Direct call to EstNLTK tokeniser that uses the best common sense tricks for tokenization
    """

    return ' '.join(word.text for word in RT_MORPH_ANALYZER(Text(text))['words'])

# Example outputs
print(rt_normalise_text('Presidendi ametiraha seadus'))
print(rt_normalise_text('Rahvusvahelise tööorganisatsiooni (ILO) konventsiooni "Mees- ja naistöötajate ... töö eest" ratifitseerimise seadus'))
print(rt_normalise_text('Protokolli nr 15 ˮInimõiguste ja põhivabaduste kaitse konventsiooni muutmise kohtaˮ ratifitseerimise seadus'))
print(rt_normalise_text('Välisriigi ... tunnistuse (apostille ’) andmise registri asutamine ja registri pidamise põhimäärus'))
print(rt_normalise_text('Osoonikihti ... protokolli 1997. aasta 15.-17. septembril Montrealis tehtud paranduste ratifitseerimise seadus'))
print(rt_normalise_text('Nõuded veise pidamise ja selleks ettenähtud ruumi või ehitise kohta﻿	'))
print(rt_normalise_text('Riigikaitseliste kohustuste täitmisega seotud õiglase hüvitise maksmise ‎tingimused ja kord'))

Presidendi ametiraha seadus
Rahvusvahelise tööorganisatsiooni ( ILO ) konventsiooni " Mees- ja naistöötajate ... töö eest " ratifitseerimise seadus
Protokolli nr 15 ˮ Inimõiguste ja põhivabaduste kaitse konventsiooni muutmise kohta ˮ ratifitseerimise seadus
Välisriigi ... tunnistuse ( apostille ’ ) andmise registri asutamine ja registri pidamise põhimäärus
Osoonikihti ... protokolli 1997 . aasta 15 . - 17 . septembril Montrealis tehtud paranduste ratifitseerimise seadus
Nõuded veise pidamise ja selleks ettenähtud ruumi või ehitise kohta
Riigikaitseliste kohustuste täitmisega seotud õiglase hüvitise maksmise tingimused ja kord


In [31]:
def rt_analyze_text(text: str):
    """
    Returns a four column table with columns index, wordform, lemma, sublemmas.
    There can be several rows for each word as each word is analysed separately.
    All rows with the same index correspond to the same word.
    Wordform columns is added to facilitate tokenisation debugging
    """
    morph_layer = RT_MORPH_ANALYZER(Text(text))['morph_analysis']
    return (
        DataFrame(
            [[i, span.text, lemma, tuple(subwords)] 
             for i, span in enumerate(morph_layer) 
             for  lemma, subwords in zip(span['lemma'], span['root_tokens'])],
            columns = ['index', 'wordform','lemma','sublemmas'])
        .drop_duplicates()
        .assign(sublemmas=lambda df: df['sublemmas'].map(list)))

# Example output
display(rt_analyze_text('Presidendi ametiraha seadus'))

,index,wordform,lemma,sublemmas
0,0,Presidendi,President,[President]
1,0,Presidendi,Presidend,"[Pre, sidend]"
2,0,Presidendi,president,[president]
3,1,ametiraha,ametiraha,"[ameti, raha]"
6,2,seadus,seaduma,[seadu]
7,2,seadus,seadus,[seadus]


## VII. Sisendtekstide analüüs ja tulemuste valideerimine

In [32]:
result = [None] *  len(all_documents)
for i, caption in tqdm(enumerate(all_documents['document_title']), total=len(all_documents)):
    result[i] = rt_analyze_text(caption).assign(doc_id = i)

result = concat(result, axis=0)

lemma_counts = (result
                .groupby('lemma').agg(occurence_count=('lemma', len), document_count = ('doc_id', lambda x: len(set(x))))
                .sort_values(['occurence_count', 'document_count'], ascending=False).reset_index()
                .pipe(lambda df: df[df['lemma'].str.len() > 0])
               )
display(lemma_counts.tail(15))

  0%|          | 0/58837 [00:00<?, ?it/s]

,lemma,occurence_count,document_count
23154,ümberarvestus,1,1
23155,ümberarvestustegur,1,1
23156,ümberjaotav,1,1
23157,ümberlaadimiskoha,1,1
23158,ümberlaadimiskoht,1,1
23159,ümberregistreerimata,1,1
23160,ümberregistreerimine,1,1
23161,ümbertegemine,1,1
23162,üürikorter,1,1
23163,üüritariif,1,1


In [33]:
NUMBER = '[0-9]+'
ESTONIAN_LETTER = '[a-z|öäõü|\-|žš]'
FOREIGN_LETTER = '[ôíëаa]'
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'
DASH_SYMBOLS = '[‑-−‒-]'
OTHER_SYMBOLS = '[§/−a\^+%*=]'
PUNCTUATION_MARKS = '[\\.,:;!?¿\\(\\)«»„““ˮ"‟”\[\]]'

In [34]:
words = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})+$', case=False, regex=True)
display(lemma_counts[words].sample(10))

,lemma,occurence_count,document_count
8666,Mesitaru,4,2
5887,Kalandusturg,6,6
19528,kajakas,1,1
19495,kahjuritõrjevahend,1,1
18561,ametisasutus,1,1
270,Pärnu,301,287
19667,keeletoimetaja,1,1
21600,riie,1,1
23059,õppeprogramm,1,1
13320,operatiivne,2,2


In [35]:
numbers = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER}|[,\.])+$')
numbers &= ~(lemma_counts['lemma'] == '.') & ~(lemma_counts['lemma'] == ',') 
display(lemma_counts[numbers].sample(10))

,lemma,occurence_count,document_count
10839,788,2,2
8815,46,3,3
14814,184,1,1
8816,461,3,3
14829,"2,1",1,1
14782,110,1,1
14845,249,1,1
6524,4.3,5,5
4842,31,8,8
14798,138,1,1


In [36]:
symbols = lemma_counts['lemma'].str.contains(
    f'^(?:{SUBSCRIPT_SYMBOLS}|{SUPERSCRIPT_SYMBOLS}|{DASH_SYMBOLS}|{PUNCTUATION_MARKS}|{OTHER_SYMBOLS})$')
display(lemma_counts[symbols])

,lemma,occurence_count,document_count
6,",",6491,4080
7,-,5868,5223
9,.,4580,3984
63,„,957,826
70,"""",886,365
79,“,828,733
141,”,596,482
188,a,444,433
232,»,343,275
233,«,340,274


In [37]:
letnums = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{NUMBER})+$', case=False, regex=True)
letnums &= ~words & ~numbers
display(lemma_counts[letnums].sample(10))

,lemma,occurence_count,document_count
17762,TKNE-5,1,1
14948,620-k,1,1
14781,10а,1,1
10828,500kroonine,2,2
15204,B1-alamkategooria,1,1
14903,3M,1,1
15271,CT1,1,1
10832,5aastane,2,2
17275,RAS1000,1,1
14923,500ruutmeetrine,1,1


In [38]:
lemma_counts[~symbols & ~numbers & ~words & ~letnums]

,lemma,occurence_count,document_count
2211,.-,26,26
2565,F. J. Wiedemann,21,21
7427,C. R. Jakobson,4,4
7484,J.V. veski,4,4
7580,M. Lüdigi,4,4
8821,A. H. Tammsaare,3,3
8935,Fr. R. Kreutzwald,3,3
8979,J. V. veski,3,3
10791,11/12/14,2,2
11597,O. luts,2,2
